In [ ]:
# Import necessary libraries and models
!pip install langchain-community
!pip install faiss-gpu
!pip install transformers
!pip install transformers_stream_generator
!pip install sentence-transformers
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.1 MB/s eta 0:00:00
  Preparin

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

# Step 1: Load and split documents for efficient retrieval
# Load the text file
loader = TextLoader('/content/gullivers_travels.txt')
documents = loader.load()

# Split the text into manageable chunks for efficient retrieval
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,     # Each chunk is 500 characters for manageability
    chunk_overlap=50,   # Each chunk has a 50-character overlap to retain context
    separators=["\n\n", "\n", " "]  # Split on paragraphs or lines, then spaces
)
docs = text_splitter.split_documents(documents)

In [ ]:
# Step 2: Initialize embedding model and vector store for retrieval
# Using a smaller embedding model to speed up processing while maintaining accuracy
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = FAISS.from_documents(docs, embedding_model)  # Store embeddings in FAISS for fast similarity search


<ipython-input-3-faf563b3df4b>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Step 3: Load the question-answering model for answer generation
# Using a smaller variant of T5 (flan-t5-large) to optimize for faster response generation
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto")
question_answerer = pipeline("text2text-generation", model=model, tokenizer=tokenizer)


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Step 4: Define the retrieval and answer generation function
def retrieve_and_answer(query):
    # Retrieve the top 3 most relevant chunks based on similarity to the query
    contexts = db.similarity_search(query, k=3)
    combined_context = " ".join([context.page_content for context in contexts])

    # Generate a concise answer based on the retrieved context
    prompt = f"Answer concisely: {combined_context}\nQuestion: {query}\nAnswer:"

    # Generate answer with a controlled response length for efficiency
    response = question_answerer(prompt, max_length=100, early_stopping=True)
    answer = response[0]['generated_text'].strip()
    print(f"Question: {query}\nAnswer: {answer}\n")


In [ ]:
# Step 5: Define the questions and run the pipeline
questions = [
    "What is the name of the land where Gulliver first lands and meets tiny people?",
    "How does Gulliver end up in Brobdingnag, the land of giants?",
    "Who are the Houyhnhnms in Gulliver’s Travels?",
    "What punishment does Gulliver face in Lilliput for refusing to destroy the enemy fleet?",
    "What are some of the laws that Gulliver finds strange in Lilliput?"
]

In [ ]:
# Run the pipeline for each question to retrieve and answer
for question in questions:
    retrieve_and_answer(question)

Question: What is the name of the land where Gulliver first lands and meets tiny people?
Answer: Lilliput

Question: How does Gulliver end up in Brobdingnag, the land of giants?
Answer: He is left on shore, is seized by one of the natives, and carried to a farmer's house

Question: Who are the Houyhnhnms in Gulliver’s Travels?
Answer: degenerate and brutal nature

Question: What punishment does Gulliver face in Lilliput for refusing to destroy the enemy fleet?
Answer: death

Question: What are some of the laws that Gulliver finds strange in Lilliput?
Answer: reward and punishment

